In [22]:
!pip install opencv-python

     --------------------------------------- 35.6/35.6 MB 11.1 MB/s eta 0:00:00


In [24]:
!pip install torchvision

     ---------------------------------------- 1.1/1.1 MB 4.2 MB/s eta 0:00:00
     -------------------------------------- 161.8/161.8 MB 7.7 MB/s eta 0:00:00


In [26]:
# Imports
import glob
import os
import numpy as np
from matplotlib import pyplot as plt
import scipy.io.wavfile as wav
from numpy.lib import stride_tricks
from PIL import Image
import pandas as pd
import cv2
import shutil
import librosa
from tqdm import tqdm
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.keras.models import Sequential,Model
from tensorflow.python.keras.layers import Dense,Dropout,Activation,Flatten,Lambda
from tensorflow.python.keras.callbacks import ModelCheckpoint
from datetime import datetime
import torchvision.transforms as transforms
from os import listdir


from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

In [27]:
df = pd.read_csv("C:/Users/Meltem/Desktop/Urban-Sound-Classification-using-CNN-main/Urban-Sound-Classification-using-CNN-main/metadata/UrbanSound8K.csv")
df.shape

(8732, 8)

In [28]:
df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [29]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
fsID,8732.0,116033.493816,57991.017218,344.000000,69942.250000,118279.000000,166942.000000,209992.000000
start,8732.0,38.645409,74.292126,0.000000,3.000000,10.376492,35.131372,600.125356
end,8732.0,42.253312,74.369669,0.105962,6.839398,14.000000,38.866979,604.125356
salience,8732.0,1.347000,0.476043,1.000000,1.000000,1.000000,2.000000,2.000000
fold,8732.0,5.385937,2.846820,1.000000,3.000000,5.000000,8.000000,10.000000
classID,8732.0,4.592877,2.894544,0.000000,2.000000,4.000000,7.000000,9.000000


In [30]:
appended = []
for i in range(1,11):
    appended.append(df[df.fold == i]['class'].value_counts())
    
class_distribution = pd.DataFrame(appended)
class_distribution = class_distribution.reset_index()
class_distribution['index'] = ["fold"+str(x) for x in range(1,11)]
class_distribution

,index,jackhammer,dog_bark,children_playing,street_music,air_conditioner,drilling,engine_idling,siren,car_horn,gun_shot
0,fold1,120,100,100,100,100,100,96,86,36,35
1,fold2,120,100,100,100,100,100,100,91,42,35
2,fold3,120,100,100,100,100,100,107,119,43,36
3,fold4,120,100,100,100,100,100,107,166,59,38
4,fold5,120,100,100,100,100,100,107,71,98,40
5,fold6,68,100,100,100,100,100,107,74,28,46
6,fold7,76,100,100,100,100,100,106,77,28,51
7,fold8,78,100,100,100,100,100,88,80,30,30
8,fold9,82,100,100,100,100,100,89,82,32,31
9,fold10,96,100,100,100,100,100,93,83,33,32


In [31]:
image_list = []
number_of_elements_on_each_file = np.empty(10, dtype='int')
total_number = 0
temp = 0

for i in range(0, 10):
  for images in os.listdir(path = f'C:/Users/Meltem/Desktop/Urban-Sound-Classification-using-CNN-main/Urban-Sound-Classification-using-CNN-main/spectrograms/{i}'): 
    # check if the image ends with png
    if (images.endswith(".png")):
        image_list.append(images)
    
  total_number = len(image_list) - temp
  print(total_number, len(image_list) ,temp) 
  number_of_elements_on_each_file[i] = total_number
  temp =  len(image_list)

1000 1000 0
429 1429 1000
1000 2429 1429
1000 3429 2429
1000 4429 3429
1000 5429 4429
374 5803 5429
1000 6803 5803
929 7732 6803
1000 8732 7732


In [32]:
size_of_image_list = len(image_list)
print(size_of_image_list)

8732


In [33]:
for i in number_of_elements_on_each_file:
  print(i)

1000
429
1000
1000
1000
1000
374
1000
929
1000


In [34]:
def decide_func(num):
  if i < 1000:
    return 0
  elif i >= 1000 and i < 1429:
    return 1
  elif i >= 1429 and i < 2429:
    return 2   
  elif i >= 2429 and i < 3429:
    return 3
  elif i >= 3429 and i < 4429:
    return 4
  elif i >= 4429 and i < 5429:
    return 5
  elif i >= 5429 and i < 5803:
    return 6
  elif i >= 5803 and i < 6803:
    return 7
  elif i >= 6803 and i < 7732:
    return 8
  elif i >= 7732 and i < 8732:
    return 9

In [35]:
def decide_func_for_final(num):
  if i < 1000:
    return 'air_conditioner'
  elif i >= 1000 and i < 1429:
    return 'car_horn'
  elif i >= 1429 and i < 2429:
    return 'children_playing'   
  elif i >= 2429 and i < 3429:
    return 'dog_bark'
  elif i >= 3429 and i < 4429:
    return 'drilling'
  elif i >= 4429 and i < 5429:
    return 'engine_idling'
  elif i >= 5429 and i < 5803:
    return 'gun_shot'
  elif i >= 5803 and i < 6803:
    return 'jackhammer'
  elif i >= 6803 and i < 7732:
    return 'siren'
  elif i >= 7732 and i < 8732:
    return 'street_music'

In [37]:
for i in range(0,size_of_image_list):
  j = decide_func(i)
  path = f"C:/Users/Meltem/Desktop/Urban-Sound-Classification-using-CNN-main/Urban-Sound-Classification-using-CNN-main/spectrograms/{j}"
  text_file = glob.glob(path + f"/**/*{image_list[i]}", recursive = True)

  image = cv2.imread(text_file[0])
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  print(f"{image_list[i]} image is writing to the grayscale file to {j} subset, # of the image is {i}")
  cv2.imwrite(f"C:/Users/Meltem/Desktop/Urban-Sound-Classification-using-CNN-main/Urban-Sound-Classification-using-CNN-main/grayscale/{j}/{image_list[i]}", gray_image)

100852-0-0-0.png image is writing to the grayscale file to 0 subset, # of the image is 0
100852-0-0-1.png image is writing to the grayscale file to 0 subset, # of the image is 1
100852-0-0-10.png image is writing to the grayscale file to 0 subset, # of the image is 2
100852-0-0-11.png image is writing to the grayscale file to 0 subset, # of the image is 3
100852-0-0-12.png image is writing to the grayscale file to 0 subset, # of the image is 4
100852-0-0-13.png image is writing to the grayscale file to 0 subset, # of the image is 5
100852-0-0-14.png image is writing to the grayscale file to 0 subset, # of the image is 6
100852-0-0-15.png image is writing to the grayscale file to 0 subset, # of the image is 7
100852-0-0-16.png image is writing to the grayscale file to 0 subset, # of the image is 8
100852-0-0-17.png image is writing to the grayscale file to 0 subset, # of the image is 9
100852-0-0-18.png image is writing to the grayscale file to 0 subset, # of the image is 10
100852-0-0-

13230-0-0-25.png image is writing to the grayscale file to 0 subset, # of the image is 103
13230-0-0-26.png image is writing to the grayscale file to 0 subset, # of the image is 104
13230-0-0-27.png image is writing to the grayscale file to 0 subset, # of the image is 105
13230-0-0-28.png image is writing to the grayscale file to 0 subset, # of the image is 106
13230-0-0-3.png image is writing to the grayscale file to 0 subset, # of the image is 107
13230-0-0-5.png image is writing to the grayscale file to 0 subset, # of the image is 108
13230-0-0-6.png image is writing to the grayscale file to 0 subset, # of the image is 109
13230-0-0-7.png image is writing to the grayscale file to 0 subset, # of the image is 110
13230-0-0-8.png image is writing to the grayscale file to 0 subset, # of the image is 111
13230-0-0-9.png image is writing to the grayscale file to 0 subset, # of the image is 112
134717-0-0-0.png image is writing to the grayscale file to 0 subset, # of the image is 113
13471

146714-0-0-36.png image is writing to the grayscale file to 0 subset, # of the image is 198
146714-0-0-38.png image is writing to the grayscale file to 0 subset, # of the image is 199
146714-0-0-41.png image is writing to the grayscale file to 0 subset, # of the image is 200
146714-0-0-43.png image is writing to the grayscale file to 0 subset, # of the image is 201
146714-0-0-45.png image is writing to the grayscale file to 0 subset, # of the image is 202
146714-0-0-47.png image is writing to the grayscale file to 0 subset, # of the image is 203
146714-0-0-49.png image is writing to the grayscale file to 0 subset, # of the image is 204
146714-0-0-50.png image is writing to the grayscale file to 0 subset, # of the image is 205
146714-0-0-53.png image is writing to the grayscale file to 0 subset, # of the image is 206
146714-0-0-55.png image is writing to the grayscale file to 0 subset, # of the image is 207
146714-0-0-56.png image is writing to the grayscale file to 0 subset, # of the i

162103-0-0-11.png image is writing to the grayscale file to 0 subset, # of the image is 297
162103-0-0-12.png image is writing to the grayscale file to 0 subset, # of the image is 298
162103-0-0-13.png image is writing to the grayscale file to 0 subset, # of the image is 299
162103-0-0-14.png image is writing to the grayscale file to 0 subset, # of the image is 300
162103-0-0-15.png image is writing to the grayscale file to 0 subset, # of the image is 301
162103-0-0-16.png image is writing to the grayscale file to 0 subset, # of the image is 302
162103-0-0-17.png image is writing to the grayscale file to 0 subset, # of the image is 303
162103-0-0-2.png image is writing to the grayscale file to 0 subset, # of the image is 304
162103-0-0-3.png image is writing to the grayscale file to 0 subset, # of the image is 305
162103-0-0-4.png image is writing to the grayscale file to 0 subset, # of the image is 306
162103-0-0-5.png image is writing to the grayscale file to 0 subset, # of the image

177621-0-0-54.png image is writing to the grayscale file to 0 subset, # of the image is 388
177621-0-0-57.png image is writing to the grayscale file to 0 subset, # of the image is 389
177621-0-0-80.png image is writing to the grayscale file to 0 subset, # of the image is 390
177621-0-0-88.png image is writing to the grayscale file to 0 subset, # of the image is 391
177621-0-0-91.png image is writing to the grayscale file to 0 subset, # of the image is 392
177621-0-0-93.png image is writing to the grayscale file to 0 subset, # of the image is 393
177726-0-0-1.png image is writing to the grayscale file to 0 subset, # of the image is 394
177726-0-0-10.png image is writing to the grayscale file to 0 subset, # of the image is 395
177726-0-0-11.png image is writing to the grayscale file to 0 subset, # of the image is 396
177726-0-0-12.png image is writing to the grayscale file to 0 subset, # of the image is 397
177726-0-0-13.png image is writing to the grayscale file to 0 subset, # of the im

...

202334-9-0-137.png image is writing to the grayscale file to 9 subset, # of the image is 8290
202334-9-0-202.png image is writing to the grayscale file to 9 subset, # of the image is 8291
202334-9-0-63.png image is writing to the grayscale file to 9 subset, # of the image is 8292
202334-9-0-88.png image is writing to the grayscale file to 9 subset, # of the image is 8293
203424-9-0-15.png image is writing to the grayscale file to 9 subset, # of the image is 8294
203424-9-0-18.png image is writing to the grayscale file to 9 subset, # of the image is 8295
203424-9-0-26.png image is writing to the grayscale file to 9 subset, # of the image is 8296
203424-9-0-34.png image is writing to the grayscale file to 9 subset, # of the image is 8297
203424-9-0-4.png image is writing to the grayscale file to 9 subset, # of the image is 8298
203424-9-0-46.png image is writing to the grayscale file to 9 subset, # of the image is 8299
203654-9-0-15.png image is writing to the grayscale file to 9 subset,

41364-9-0-27.png image is writing to the grayscale file to 9 subset, # of the image is 8392
42953-9-0-16.png image is writing to the grayscale file to 9 subset, # of the image is 8393
42953-9-0-21.png image is writing to the grayscale file to 9 subset, # of the image is 8394
42953-9-0-28.png image is writing to the grayscale file to 9 subset, # of the image is 8395
42953-9-0-37.png image is writing to the grayscale file to 9 subset, # of the image is 8396
42953-9-0-49.png image is writing to the grayscale file to 9 subset, # of the image is 8397
42953-9-0-50.png image is writing to the grayscale file to 9 subset, # of the image is 8398
42954-9-0-12.png image is writing to the grayscale file to 9 subset, # of the image is 8399
42954-9-0-18.png image is writing to the grayscale file to 9 subset, # of the image is 8400
42954-9-0-2.png image is writing to the grayscale file to 9 subset, # of the image is 8401
42954-9-0-20.png image is writing to the grayscale file to 9 subset, # of the ima

66599-9-0-17.png image is writing to the grayscale file to 9 subset, # of the image is 8495
66599-9-1-11.png image is writing to the grayscale file to 9 subset, # of the image is 8496
66599-9-1-21.png image is writing to the grayscale file to 9 subset, # of the image is 8497
66599-9-1-23.png image is writing to the grayscale file to 9 subset, # of the image is 8498
66599-9-1-4.png image is writing to the grayscale file to 9 subset, # of the image is 8499
69304-9-0-0.png image is writing to the grayscale file to 9 subset, # of the image is 8500
69304-9-0-12.png image is writing to the grayscale file to 9 subset, # of the image is 8501
69304-9-0-14.png image is writing to the grayscale file to 9 subset, # of the image is 8502
69304-9-0-15.png image is writing to the grayscale file to 9 subset, # of the image is 8503
69304-9-0-6.png image is writing to the grayscale file to 9 subset, # of the image is 8504
69304-9-0-7.png image is writing to the grayscale file to 9 subset, # of the image 

78326-9-0-6.png image is writing to the grayscale file to 9 subset, # of the image is 8591
78326-9-0-7.png image is writing to the grayscale file to 9 subset, # of the image is 8592
79377-9-0-10.png image is writing to the grayscale file to 9 subset, # of the image is 8593
79377-9-0-12.png image is writing to the grayscale file to 9 subset, # of the image is 8594
79377-9-0-13.png image is writing to the grayscale file to 9 subset, # of the image is 8595
79377-9-0-2.png image is writing to the grayscale file to 9 subset, # of the image is 8596
79377-9-0-4.png image is writing to the grayscale file to 9 subset, # of the image is 8597
79377-9-0-8.png image is writing to the grayscale file to 9 subset, # of the image is 8598
83191-9-0-0.png image is writing to the grayscale file to 9 subset, # of the image is 8599
83191-9-0-11.png image is writing to the grayscale file to 9 subset, # of the image is 8600
83191-9-0-13.png image is writing to the grayscale file to 9 subset, # of the image is

98202-9-0-7.png image is writing to the grayscale file to 9 subset, # of the image is 8687
98202-9-1-20.png image is writing to the grayscale file to 9 subset, # of the image is 8688
98202-9-1-27.png image is writing to the grayscale file to 9 subset, # of the image is 8689
98202-9-1-5.png image is writing to the grayscale file to 9 subset, # of the image is 8690
98263-9-0-1.png image is writing to the grayscale file to 9 subset, # of the image is 8691
98263-9-0-10.png image is writing to the grayscale file to 9 subset, # of the image is 8692
98263-9-0-22.png image is writing to the grayscale file to 9 subset, # of the image is 8693
98263-9-0-24.png image is writing to the grayscale file to 9 subset, # of the image is 8694
98263-9-0-33.png image is writing to the grayscale file to 9 subset, # of the image is 8695
98263-9-0-35.png image is writing to the grayscale file to 9 subset, # of the image is 8696
98680-9-0-11.png image is writing to the grayscale file to 9 subset, # of the image

In [42]:
import cv2

In [50]:
# Python program to explain cv2.imshow() method  
# importing cv2 import cv2  
# path path = r'C:\Users\Rajnish\Desktop\geeksforgeeks.png' 
# Reading an image in default mode image = cv2.imread(path) 
# Window name in which image is displayed window_name = 'image'
  
# Using cv2.imshow() method 
# Displaying the image cv2.imshow(window_name, image)

print('Result of the spectogram')
path1 = r"C:/Users/Meltem/Desktop/Urban-Sound-Classification-using-CNN-main/Urban-Sound-Classification-using-CNN-main/spectrograms/3/100795-3-1-1.png"
image1 = cv2.imread(path1)
window_name1 = 'spectrogram'
cv2.imshow(window_name1,image1)


Result of the spectogram


In [51]:
print('Result of the grayscale')
path2 = r'C:/Users/Meltem/Desktop/Urban-Sound-Classification-using-CNN-main/Urban-Sound-Classification-using-CNN-main/grayscale/3/100795-3-1-1.png'
image2 = cv2.imread(path2)
window_name2 = 'grayscale'
cv2.imshow(window_name2,image2)


Result of the grayscale


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:967: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [ ]:
print('Result of the resizing')
img = cv2.imread("C:/Users/Meltem/Desktop/Urban-Sound-Classification-using-CNN-main/Urban-Sound-Classification-using-CNN-main/resizing/3/100795-3-1-1.png")
cv2.imshow('image3',img)

print('Result of the normalization')
img = cv2.imread("C:/Users/Meltem/Desktop/Urban-Sound-Classification-using-CNN-main/Urban-Sound-Classification-using-CNN-main/normalizasyon/3/100795-3-1-1.png")
cv2.imshow('image4',img)